In [8]:
import os # this allows simple navigation of folders in the notebook
import pandas as pd # this loads Pandas
import numpy as np # This loads NumPy, used to do maths in the notebook
from ebmdatalab import bq # this loads the bennett institute bigquery function
import glob

In [3]:
# get prescribing data - for reference only - this won't work unless you have a BQ query account
sql = '''
SELECT DATE(month) AS month, pct, practice, bnf_name, bnf_code, items, quantity, net_cost, actual_cost
FROM hscic.normalised_prescribing
WHERE
(bnf_code like '0101%' or bnf_code like '0103%')
AND
month >='2019-01-01'
'''
tempfile = os.path.join("..","data","omep_df_temp.csv") 
omep_df = bq.cached_read(sql, csv_path=tempfile, use_cache=True)

# chunk data go <100mb for GitHub
import pandas as pd
import os

# Define the maximum compressed file size in bytes (100 MB)
max_compressed_file_size = 100 * 1024 * 1024

# Initialize variables
file_index = 0
total_rows = len(omep_df)
rows_per_chunk = total_rows // 4  # Start with an approximate number of rows per file

# Start writing the DataFrame to compressed CSV files
for start in range(0, total_rows, rows_per_chunk):
    while True:
        # Select a chunk of the DataFrame
        omep_df_chunk = omep_df.iloc[start:start + rows_per_chunk]

        # Save the chunk to a compressed CSV file
        file_name = os.path.join("..","data",f"omep_df_chunk_{file_index}.csv.gz")
        omep_df_chunk.to_csv(file_name, index=False, compression='gzip')

        # Check the size of the compressed file
        compressed_file_size = os.path.getsize(file_name)

        # If the file size is under the limit, proceed to the next chunk
        if compressed_file_size < max_compressed_file_size:
            print(f"Saved {file_name}, size: {compressed_file_size / (1024 * 1024):.2f} MB")
            file_index += 1
            break
        else:
            # If the file size exceeds the limit, reduce the chunk size and try again
            rows_per_chunk = rows_per_chunk // 2

            # Remove the oversized file to try again
            os.remove(file_name)

            # Check if rows_per_chunk has become too small (to avoid infinite loop)
            if rows_per_chunk == 0:
                raise ValueError("Cannot compress to below 100 MB, consider a larger chunk size.")


#delete the temp file
os.remove(file_name)

Saved ../data/data_chunk_0.csv.gz, size: 41.37 MB
Saved ../data/data_chunk_1.csv.gz, size: 41.57 MB
Saved ../data/data_chunk_2.csv.gz, size: 41.36 MB
Saved ../data/data_chunk_3.csv.gz, size: 41.60 MB
Saved ../data/data_chunk_4.csv.gz, size: 0.00 MB


In [7]:
# get related data sets
#BNF data
sql = 'SELECT * FROM hscic.bnf'
exportfile = os.path.join("..","data","bnf_df.csv") 
bnf_df = bq.cached_read(sql, csv_path=exportfile, use_cache=False)

#Practice data
sql = 'SELECT * FROM hscic.practices'
exportfile = os.path.join("..","data","practices_df.csv")
practices_df = bq.cached_read(sql, csv_path=exportfile, use_cache=False)

# Practice stats data
sql = 'SELECT DATE(month) AS month, total_list_size, practice FROM hscic.practice_statistics'
exportfile = os.path.join("..","data","statistics_df.csv") 
statistics_df = bq.cached_read(sql, csv_path=exportfile, use_cache=False)

# CCG/SICBL data
sql = 'SELECT * from hscic.ccgs'
exportfile = os.path.join("..","data","ccg_df.csv") 
statistics_df = bq.cached_read(sql, csv_path=exportfile, use_cache=False)

Downloading: 100%|█████████████████████████████████████████████████████████████|
Downloading: 100%|█████████████████████████████████████████████████████████████|
Downloading: 100%|█████████████████████████████████████████████████████████████|
Downloading: 100%|█████████████████████████████████████████████████████████████|
